### Baseline RandomForest Model: 1) Throw everything in 2) Log features 3) Cross features

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# from sklearn.compose import ColumnTransformer

In [2]:
data = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

## Minimum Feature Engineering

In [3]:
# Drop unhelpful cat features
# cat0, cat2, cat4, cat6, cat7 cuz of too much skew
def make_train(data):
    train = data.drop(['cat0', 'cat2', 'cat4', 'cat6', 'cat7'], axis=1)
    
    for col in train.select_dtypes(include='object'):
        le = LabelEncoder()
        train[col] = le.fit_transform(train[col])
    
    return train
#consider removing outlier categories (ex. A nd C for cat5 )
#todo: figure out how to use labelencoder in pipeline

In [4]:
train = make_train(data)

In [5]:
train.shape

(300000, 21)

In [6]:
rf = RandomForestRegressor(max_depth=5)
X = train.drop(['id', 'target'], axis=1)
y = train.target

scores = cross_val_score(rf, X, y, cv=5, scoring='neg_root_mean_squared_error', verbose=1, n_jobs=-1)
print(scores.mean)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  3.0min remaining:  4.5min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  3.0min finished


<built-in method mean of numpy.ndarray object at 0x000001DB23D83300>


In [7]:
scores.mean()

-0.8695958663409027

In [8]:
X.shape

(300000, 19)

In [9]:
scores

array([-0.86811497, -0.86930236, -0.87048567, -0.87092638, -0.86914995])

In [10]:
test = make_train(test)
test_X = test.drop(['id'], axis=1)

In [11]:
def predict_test(model, X, y):
    model.fit(X, y)
    pred = model.predict(test_X)
#     pred.to_csv('test_pred.csv')
    return pred

In [12]:
pred = predict_test(rf, X, y)
pred_df = pd.DataFrame(test['id'], columns=['id'])
pred_df['target'] = pred
pred_df.to_csv('results.csv', index=False)

In [13]:
# !kaggle competitions submit -c tabular-playground-series-feb-2021 -f results.csv -m "baseline rf minimal processing 1"